In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NATHAN\\Text-Summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\NATHAN\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\NATHAN\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)

        #Loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=float(self.config.num_train_epochs),
            per_device_train_batch_size=int(self.config.per_device_train_batch_size), per_device_eval_batch_size=int(self.config.per_device_train_batch_size),
            weight_decay=float(self.config.weight_decay), logging_steps=int(self.config.logging_steps), 
            eval_steps=int(self.config.eval_steps), save_steps=int(self.config.save_steps),
            gradient_accumulation_steps=int(self.config.gradient_accumulation_steps)
         
        )

        trainer = Trainer(model=model_t5, args=trainer_args,
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt['train'],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()

        ## Save the model
        model_t5.save_pretrained(os.path.join(self.config.root_dir,"t5-samsum-model"))
        ## Save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-12-11 22:50:01,930: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-11 22:50:01,943: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-11 22:50:01,943: INFO: common: created directory at: artifacts]


C:\Users\NATHAN\AppData\Local\Temp\ipykernel_4536\2902391237.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_t5, args=trainer_args,


Step,Training Loss
10,3.058800
20,2.721700
30,2.527700
40,2.426900
50,2.269800
60,2.260700
70,2.268600
80,2.222800
90,2.214400
100,2.231400
